#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader=WebBaseLoader("https://docs.smith.langchain.com/observability/how_to_guides/tracing/log_traces_to_project")
loader

In [6]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/tracing/log_traces_to_project', 'title': 'Log traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'You can change the destination project of your traces both statically through environment variables and dynamically at runtime.', 'language': 'en'}, page_content='\n\n\n\n\nLog traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityTutorialsAdd observability to your LLM applicationHow-to GuidesTracingAnnotate code for tracingToggle tracing on and offUpload files with tracesLog traces to specific projectSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog multimodal trac

In [7]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [8]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/tracing/log_traces_to_project', 'title': 'Log traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'You can change the destination project of your traces both statically through environment variables and dynamically at runtime.', 'language': 'en'}, page_content='Log traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/tracing/log_traces_to_project', 'title': 'Log traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'You can change the destination project of your traces both statically through environment variables and dynamically at runtime.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to A

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [11]:
vectorstoredb

In [12]:
### Query From a vectordb
query="How to set the destination"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'You can change the destination project of your traces both statically through environment variables and dynamically at runtime.\nSet the destination project statically‚Äã\nAs mentioned in the Tracing Concepts section, LangSmith uses the concept of a Project to group traces. If left unspecified, the project is set to default. You can set the LANGCHAIN_PROJECT environment variable to configure a custom project name for an entire application run. This should be done before executing your application.\nexport LANGCHAIN_PROJECT=my-custom-project\nIf the project specified does not exist, it will be created automatically when the first trace is ingested.\nSet the destination project dynamically‚Äã\nYou can also set the project name at program runtime in various ways, depending on how you are annotating your code for tracing. This is useful when you want to log traces to different projects within the same application.'

In [13]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [14]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C8E7922EC0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C8E727B130>, root_client=<openai.OpenAI object at 0x000001C8BB936B60>, root_async_client=<openai.AsyncOpenAI object at 0x000001C8E7923B20>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_do

In [15]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"How to set the destination",
    "context":[Document(page_content="You can change the destination project of your traces both statically through environment variables and dynamically at runtime.\nSet the destination project statically‚Äã\nAs mentioned in the Tracing Concepts section, LangSmith uses the concept of a Project to group traces. If left unspecified, the project is set to default. You can set the LANGCHAIN_PROJECT environment variable to configure a custom project name for an entire application run. This should be done before executing your application.\nexport LANGCHAIN_PROJECT=my-custom-project\nIf the project specified does not exist, it will be created automatically when the first trace is ingested. ")]
})

'To change the destination project of your traces statically, you need to set the `LANGCHAIN_PROJECT` environment variable to your desired project name before executing your application. For example, you can run the command `export LANGCHAIN_PROJECT=my-custom-project`. If the specified project does not exist, it will be created automatically when the first trace is ingested.'

In [16]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [17]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [18]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C8E7921CF0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [19]:
### Get the response form the LLM
response=retrieval_chain.invoke({"input":"How to set the destination"})
response['answer']

'To set the destination project for your traces dynamically at runtime, you can use various methods depending on how you are annotating your code for tracing. This allows you to log traces to different projects within the same application. When you set the project name dynamically, it overrides any project name set by the LANGCHAIN_PROJECT environment variable.'

In [20]:
response

{'input': 'How to set the destination',
 'context': [Document(id='aa9d5352-e117-4506-916f-ba1c186a398b', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/tracing/log_traces_to_project', 'title': 'Log traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'You can change the destination project of your traces both statically through environment variables and dynamically at runtime.', 'language': 'en'}, page_content='You can change the destination project of your traces both statically through environment variables and dynamically at runtime.\nSet the destination project statically‚Äã\nAs mentioned in the Tracing Concepts section, LangSmith uses the concept of a Project to group traces. If left unspecified, the project is set to default. You can set the LANGCHAIN_PROJECT environment variable to configure a custom project name for an entire application run. This should be done before executing your application.\nexport LANGCHAIN_P

In [21]:
response['context']

[Document(id='aa9d5352-e117-4506-916f-ba1c186a398b', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/tracing/log_traces_to_project', 'title': 'Log traces to specific project | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'You can change the destination project of your traces both statically through environment variables and dynamically at runtime.', 'language': 'en'}, page_content='You can change the destination project of your traces both statically through environment variables and dynamically at runtime.\nSet the destination project statically‚Äã\nAs mentioned in the Tracing Concepts section, LangSmith uses the concept of a Project to group traces. If left unspecified, the project is set to default. You can set the LANGCHAIN_PROJECT environment variable to configure a custom project name for an entire application run. This should be done before executing your application.\nexport LANGCHAIN_PROJECT=my-custom-project\nIf the project specified d